In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [7]:
import numpy as np
import pandas as pd
from kcsd import KCSD1D

In [8]:
from ecephys.sglx_utils import load_trigger
from ecephys.plot import lfp_explorer, colormesh_timeseries_explorer
from ecephys.signal.utils import mean_subtract, median_subtract
from ecephys.signal.csd import get_kcsd
import ecephys_analyses as ea

In [9]:
def mm_to_mask(lo, hi, ele_pos):
    assert lo < hi, "Lower bound must be less than upper bound."     
    return np.logical_and(ele_pos >= lo, ele_pos <= hi).flatten()

def mm_to_idx(lo, hi, ele_pos):
    idx = np.where(millimeters_to_mask(lo, hi, ele_pos))[0]
    return idx

def mm_to_chans(lo, hi, ele_pos, chans):
    return chans[mm_to_mask(lo, hi, ele_pos)]

In [8]:
SUBJECT = "Segundo"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 1680 # pick a 1 second window that has SPW-Rs

In [31]:
SUBJECT = "Segundo"
EXPERIMENT = "isoflurane"
CONDITION = "all"
START_TIME = 2 # pick a 1 second window that has SPW-Rs

In [9]:
SUBJECT = "Valentino"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 3880 # pick a 1 second window that has SPW-Rs

In [10]:
SUBJECT = "Valentino"
EXPERIMENT = "sevo-dex"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 355 # pick a 1 second window that has SPW-Rs

In [19]:
# During Dex-induced theta, in 2-25-2020_g0_t1

SUBJECT = "Valentino"
EXPERIMENT = "sevo-dex"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 2986 # pick a 1 second window that has SPW-Rs

In [10]:
SUBJECT = "Valentino"
EXPERIMENT = "atropine"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 828 # pick a 1 second window that has SPW-Rs

In [15]:
SUBJECT = "Valentino"
EXPERIMENT = "isoflurane"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 83 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.8
hpc_hi = 6.1
wm_center = 6.12
wm_thickness = 0.133

In [66]:
SUBJECT = "Valentino"
EXPERIMENT = "sevoflurane"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 77 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.866
hpc_hi = 6.166
wm_center = 6.36
wm_thickness = 0.133

In [97]:
SUBJECT = "Valentino"
EXPERIMENT = "ketamine-90mg"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 16 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.856
hpc_hi = 6.156
wm_center = 6.28
wm_thickness = 0.133

In [48]:
SUBJECT = "Doppio"
EXPERIMENT = "isoflurane"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 77 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.8
hpc_hi = 5.8
wm_center = 5.9
wm_thickness = 0.133

In [77]:
SUBJECT = "Doppio"
EXPERIMENT = "sevoflurane"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 202 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.8
hpc_hi = 6.0
wm_center = 5.94
wm_thickness = 0.133

In [139]:
SUBJECT = "Doppio"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 1708 # pick a 1 second window that has SPW-Rs

In [8]:
SUBJECT = "Doppio"
EXPERIMENT = "atropine-50mg"
CONDITION = "all"
START_TIME = 1159 # pick a 1 second window that has SPW-Rs

In [189]:
SUBJECT = "Alessandro"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 1766 # pick a 1 second window that has SPW-Rs

In [215]:
SUBJECT = "Eugene"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 2600 # pick a 1 second window that has SPW-Rs

In [239]:
SUBJECT = "Allan"
EXPERIMENT = "sleep-homeostasis"
CONDITION = "recovery-sleep"
START_TIME = 2007 # pick a 1 second window that has SPW-Rs

In [7]:
SUBJECT = "Allan"
EXPERIMENT = "dexmedetomidine"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 3 # pick a 1 second window that has SPW-Rs

In [8]:
SUBJECT = "Allan"
EXPERIMENT = "atropine"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 35 # pick a 1 second window that has SPW-Rs

In [106]:
SUBJECT = "Allan"
EXPERIMENT = "ketamine"
CONDITION = "all"
PROBE = "imec0"
START_TIME = 1791 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.2
hpc_hi = 5.4
wm_center = 5.48
wm_thickness = 0.133

In [39]:
SUBJECT = "Doppio"
EXPERIMENT = "ketamine-120mg"
CONDITION = "all"
PROBE = "imec1"
START_TIME = 14 # pick a 1 second window that has SPW-Rs

hpc_lo = 3.3
hpc_hi = 5.066
wm_center = 5.2
wm_thickness = 0.133

## 1D kCSD on whole probe

In [11]:
chans = ea.get_channels(SUBJECT, EXPERIMENT, PROBE, "all")
bin_path = ea.get_sglx_style_datapaths(subject=SUBJECT, experiment=EXPERIMENT, condition=CONDITION, ext="lf.bin")[0]
sig = load_trigger(bin_path, chans, start_time=(START_TIME), end_time=(START_TIME + 1.0))
sig.values = mean_subtract(sig.values)

nChan: 385, nFileSamp: 9000060


In [ ]:
electrode_pitch = 0.020
ele_pos = np.arange(0, len(sig.channel)) * electrode_pitch
csd = get_kcsd(
    sig, ele_pos, gdx=electrode_pitch, do_lcurve=True
)

In [40]:
print(f"Hippocampus: {hpc_lo}mm, {hpc_hi}mm")
hpc_chans = csd.sel(pos=slice(hpc_lo, hpc_hi)).channel.values # These are the channel numbers belonging to the hippocampus
print(f"Channels:\n {hpc_chans}")
print("")

wm_lo = wm_center - wm_thickness
wm_hi = wm_center + wm_thickness
print(f"White matter: {wm_lo}mm, {wm_hi}mm")
wm_chans = csd.sel(pos=slice(wm_lo, wm_hi)).channel.values # These are the channel numbers belonging to the hippocampus
print(f"Channels:\n {wm_chans}")

Hippocampus: 3.3mm, 5.066mm
Channels:
 [331 332 335 336 339 340 343 344 347 348 351 352 355 356 359 360 363 364
 367 368 371 372 375 376 379 380 383   1   2   5   6   9  10  13  14  17
  18  21  22  25  26  29  30  33  34  37  38  41  42  45  46  49  50  53
  54  57  58  61  62  65  66  69  70  73  74  77  78  81  82  85  86  89
  90  93  94  97  98 101 102 105 106 109 110 113 114 117 118 121 122]

White matter: 5.067mm, 5.333mm
Channels:
 [125 126 129 130 133 134 137 138 141 142 145 146 149]


In [41]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(csd.time.values, csd.kcsd.estm_x, csd.values, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")
ax.set_title(f"{SUBJECT}, {EXPERIMENT}, {bin_path.stem} ({hpc_lo}mm - {hpc_hi}mm), ({wm_lo}mm - {wm_hi}mm)")
ax.axhline(hpc_hi, alpha=0.5, color='dimgrey', linestyle="--")
ax.axhline(hpc_lo, alpha=0.5, color='dimgrey', linestyle="--")
ax.axhline(wm_hi, alpha=0.5, color='brown', linestyle="--")
ax.axhline(wm_lo, alpha=0.5, color='brown', linestyle="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
hpc_chans

array([331, 332, 335, 336, 339, 340, 343, 344, 347, 348, 351, 352, 355,
       356, 359, 360, 363, 364, 367, 368, 371, 372, 375, 376, 379, 380,
       383,   1,   2,   5,   6,   9,  10,  13,  14,  17,  18,  21,  22,
        25,  26,  29,  30,  33,  34,  37,  38,  41,  42,  45,  46,  49,
        50,  53,  54,  57,  58,  61,  62,  65,  66,  69,  70,  73,  74,
        77,  78,  81,  82,  85,  86,  89,  90,  93,  94,  97,  98, 101,
       102, 105, 106, 109, 110, 113, 114, 117, 118, 121, 122])